In [1]:
!pip install pympi-ling pyannote.audio

  Using cached pympi_ling-1.70.2-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached soundfile-0.13.0-py2.py3-none-manylinux_2_28_x86_64.whl.metadata (16 kB)
  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Preparing metadata (setup.py) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cach

In [ ]:
from pyannote.audio import Pipeline
from pyannote.core import Annotation
import os
import pympi


# 1) Initialiser le pipeline de VAD pyannote
pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection", use_auth_token="HF_TOKEN")

HYPER_PARAMETERS = {
# Supprime les régions de parole inférieures à cette durée (en secondes)
"min_duration_on": 0.3,
# Remplit les régions de non-parole inférieures à cette durée (en secondes)
"min_duration_off": 0.2,
"onset": 0.8
}
pipeline.instantiate(HYPER_PARAMETERS)

def apply_vad_to_eaf(
    eaf_folder, 
    audio_folder, 
    output_folder,
    audio_ext=".wav",         # Extension attendue des fichiers audio
    new_tier_name="AutoVAD"   # Nom de la tier de détection d'activité vocale
    ):
    """
    Parcourt un dossier de fichiers EAF, applique la détection d'activité vocale
    sur l'audio correspondant et ajoute une nouvelle tier au fichier EAF.
    
    :param eaf_folder: Dossier contenant les fichiers EAF.
    :param audio_folder: Dossier contenant les fichiers audio.
    :param output_folder: Dossier où enregistrer les EAF modifiés.
    :param audio_ext: Extension des fichiers audio (e.g. '.wav').
    :param new_tier_name: Nom du tier à créer dans l'EAF.
    """

    # Crée le dossier de sortie s’il n’existe pas
    os.makedirs(output_folder, exist_ok=True)

    # Parcourir tous les fichiers EAF
    for filename in os.listdir(eaf_folder):
        if filename.lower().endswith(".eaf"):
            eaf_path = os.path.join(eaf_folder, filename)
            
            # Correspondance du fichier audio par le nom de base
            base_name, _ = os.path.splitext(filename)
            audio_path = os.path.join(audio_folder, base_name + audio_ext)

            if not os.path.isfile(audio_path):
                print(f"[Attention] Pas de fichier audio pour {filename}, chemin attendu : {audio_path}")
                continue

            # Appliquer la détection d'activité vocale
            print(f"[Info] Application de la VAD sur l'audio : {audio_path}")
            vad_result = pipeline(audio_path)

            # Charger l’EAF avec pympi-ling & ajouter l'audio au fichier EAF
            eaf_obj = pympi.Elan.Eaf(eaf_path)
            eaf_obj.add_linked_file(audio_path)

            # Ajouter un nouveau tier pour la VAD (si pas déjà existant)
            if new_tier_name not in eaf_obj.get_tier_names():
                eaf_obj.add_tier(new_tier_name)

            # Parcourir les segments détectés et les ajouter sous forme d’annotations
            # Les temps de pyannote sont en secondes, ELAN attend des millisecondes.
            for segment in vad_result.get_timeline().support():
                start_time_sec = segment.start
                end_time_sec = segment.end

                # Convertir en millisecondes
                start_ms = int(start_time_sec * 1000)
                end_ms = int(end_time_sec * 1000)

                # Ajout d’une annotation (texte = "speech" ou ce que vous voulez)
                eaf_obj.add_annotation(
                    new_tier_name,
                    start_ms,
                    end_ms,
                    value="speech"
                )

            # Sauvegarder le fichier EAF modifié dans le dossier de sortie
            output_eaf_path = os.path.join(output_folder, filename)
            eaf_obj.to_file(output_eaf_path)
            print(f"[OK] Nouveau fichier EAF créé : {output_eaf_path}")


if __name__ == "__main__":
    dossier_eaf = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_VAD"
    dossier_audio = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav"
    dossier_sortie = "/home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_pyannote"

    apply_vad_to_eaf(
        eaf_folder=dossier_eaf,
        audio_folder=dossier_audio,
        output_folder=dossier_sortie,
        audio_ext=".wav",
        new_tier_name="pyannote_VAD"
    )


Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
[Info] Application de la VAD sur l'audio : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw0552.wav
[OK] Nouveau fichier EAF créé : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_pyannote/bdmw0552.eaf
[Info] Application de la VAD sur l'audio : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw0070.wav
[OK] Nouveau fichier EAF créé : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/annotations/MFA-TextGrid/raw/synth_eaf_pyannote/bdmw0070.eaf
[Info] Application de la VAD sur l'audio : /home/or-llsh-156-l01/projets/CREAM/data/Haiti-CMU/recordings/converted/synth/wav/bdmw1162.wav
[OK] Nouveau fichier EAF créé : /home/or-llsh-156-l01/proj